# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Male, VISl, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('wt/wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISl' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."


In [7]:
session_id = filtered_sessions.index.values[1]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 568


,isi_violations,isolation_distance,presence_ratio,L_ratio,silhouette_score,waveform_velocity_below,waveform_PT_ratio,peak_channel_id,waveform_spread,amplitude_cutoff,waveform_repolarization_slope,cumulative_drift,waveform_recovery_slope,nn_miss_rate,firing_rate,nn_hit_rate,max_drift,waveform_duration,local_index_unit,waveform_velocity_above,snr,d_prime,waveform_halfwidth,cluster_id,waveform_amplitude,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951873075,0.007152,217.235358,0.99,0.008822,0.102778,-0.343384,0.474643,850217391,100.0,0.005241,0.420450,109.34,-0.074171,0.068443,28.521041,0.981333,46.12,0.233501,11,-0.215841,2.372665,3.176851,0.151089,11,94.135470,NaN,400.0,2.920165,2.528816,1.203947,3.781982,0.927133,0.278908,0.031626,0.036298,0.060875,80.000,0.258968,0.111,0.02,15.0,-0.281873,-0.292035,-0.250845,-0.287273,-0.270851,135.0,120.0,0.013332,0.011072,0.011968,0.479271,5.469235e-05,-27.500,4952,0.25,27.190068,28.536524,30.375952,22.075780,31.185267,NaN,0.0555,False,False,False,False,False,0.119373,False,0.009507,0.001270,0.004365,0.028352,0.009290,240,27,757988387,23,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970319,1249.998763,True
951873097,0.001013,122.484204,0.99,0.000807,0.050535,-0.206030,0.330639,850217393,80.0,0.000001,0.591829,104.84,-0.086391,0.000000,26.797376,0.999333,64.57,0.412060,13,-0.068677,3.967552,6.785515,0.164824,13,170.762865,NaN,200.0,2.857941,2.415720,0.557588,3.520448,0.991270,0.257158,0.017766,0.008091,0.023106,85.000,0.445727,0.290,0.02,15.0,-0.320775,-0.235060,-0.144444,-0.579909,-0.262002,180.0,0.0,0.002854,0.074976,0.241898,0.035710,6.489135e-03,10.000,4969,0.25,26.866547,21.955271,29.236065,18.960631,28.983773,NaN,0.2055,False,False,False,False,False,0.035932,False,0.003915,0.000754,0.001397,0.026542,0.002959,260,43,757988387,24,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970319,1249.998763,True
951873063,0.141647,56.159655,0.99,0.006651,0.164055,-0.703119,0.608431,850217393,140.0,0.017298,0.485948,273.99,-0.116500,0.001177,2.388511,0.936720,49.35,0.219765,10,0.000000,2.573229,4.377841,0.137353,10,106.887495,NaN,5600.0,3.989308,0.956190,1.830000,1.133333,1.517370,0.452706,0.020704,0.411478,0.247698,63.214,1.174016,0.111,0.02,8.0,0.678218,0.269737,0.776145,NaN,0.772541,315.0,120.0,0.022920,0.557782,0.000147,NaN,3.605302e-08,6.429,4939,0.50,2.447853,0.586184,3.859631,0.877198,3.770182,NaN,0.0635,False,False,False,False,False,0.150271,False,0.337184,0.138462,0.066968,0.300025,0.165775,260,43,757988387,24,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970319,1249.998763,True
951873182,0.308596,70.046500,0.99,0.023951,0.082816,0.255085,0.741227,850217411,90.0,0.033559,0.452099,346.10,-0.169453,0.012990,1.618139,0.962667,35.54,0.151089,20,0.000000,3.017129,4.617693,0.096147,21,131.342640,NaN,900.0,4.943365,1.626667,4.780000,1.435897,1.598639,0.499537,0.047024,0.118419,0.201685,20.000,0.567178,0.000,0.02,8.0,NaN,-1.000000,-0.891156,0.898438

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISl') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 11


,isi_violations,isolation_distance,presence_ratio,L_ratio,silhouette_score,waveform_velocity_below,waveform_PT_ratio,peak_channel_id,waveform_spread,amplitude_cutoff,waveform_repolarization_slope,cumulative_drift,waveform_recovery_slope,nn_miss_rate,firing_rate,nn_hit_rate,max_drift,waveform_duration,local_index_unit,waveform_velocity_above,snr,d_prime,waveform_halfwidth,cluster_id,waveform_amplitude,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951879810,0.098388,71.420863,0.97,0.007994,0.044953,NaN,0.214897,850221433,70.0,0.006997,0.360398,258.55,-0.047621,0.015806,8.573389,0.967333,33.77,0.288442,287,0.220747,1.909956,4.611717,0.233501,289,82.740255,NaN,400.0,8.951232,0.773333,1.145121,0.993939,1.086451,0.506027,0.080196,0.336693,0.076406,45.000,0.934167,0.0,0.08,1.0,0.136490,0.134503,0.241517,0.437500,0.078113,180.0,0.0,0.511348,0.739475,0.103910,0.368108,0.649604,25.00,4916,0.50,4.167651,0.612829,18.082363,0.178729,18.163484,NaN,0.0515,False,False,False,False,False,0.361458,False,0.290091,0.372308,0.042965,0.876914,0.074831,2800,59,757988395,278,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.923896,1249.996829,True
951880180,0.093169,34.987615,0.99,0.049309,0.163900,0.618090,0.016580,850221463,110.0,0.046097,0.342415,274.98,-0.047216,0.008250,5.015031,0.615160,35.70,0.618090,321,0.931639,1.585524,3.320705,0.329648,323,134.854980,NaN,700.0,7.868306,1.121159,1.600000,1.127044,1.867751,0.303468,0.109979,0.190940,0.150200,17.143,0.407789,0.0,0.02,1.0,NaN,0.107456,-0.273369,NaN,0.409259,270.0,90.0,NaN,0.612244,0.199900,NaN,0.046043,40.00,5007,0.50,5.031256,4.662831,3.932175,6.711660,5.901067,NaN,0.1685,False,False,False,False,False,0.411203,False,0.211822,0.005276,0.152945,0.123633,0.113967,2940,11,757988395,293,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.923896,1249.996829,True
951880237,0.022882,60.423434,0.99,0.009443,0.083719,0.137353,41.444251,850221467,110.0,0.000471,0.125117,111.36,-0.024444,0.007293,9.943506,0.937120,49.53,1.497152,326,1.047038,1.099949,4.616915,NaN,328,247.704795,NaN,800.0,2.193496,0.792068,0.690512,0.315447,0.391323,0.155134,0.313355,0.159224,0.230359,41.250,0.632825,0.0,0.02,4.0,0.186106,0.355769,0.489362,0.354839,0.266539,180.0,60.0,0.400553,0.002341,0.000011,0.116033,0.004701,18.75,5011,0.75,7.981599,7.487174,13.926510,2.884884,8.823292,NaN,0.0725,False,False,False,False,False,0.316356,False,0.191989,0.030554,0.106884,0.307918,0.089919,2960,27,757988395,295,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.923896,1249.996829,True
951880300,0.004277,78.393248,0.99,0.000190,0.258041,0.343384,0.150658,850221473,80.0,0.000212,1.032202,132.20,-0.178882,0.000172,5.323525,0.994856,47.31,0.604355,332,0.412060,2.535009,6.099884,0.315913,334,367.470480,NaN,200.0,1.504294,1.517101,0.711989,1.357576,0.836610,0.323480,0.023486,0.488474,0.091391,10.000,0.706221,0.0,0.32,8.0,NaN,-0.040000,0.215701,-0.307692,0.519178,315.0,150.0,NaN,0.911975,0.118507,0.811198

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
951879810    18.163484
951880180     5.901067
951880237     8.823292
951880300    10.745685
951880379     2.656446
951880367     7.850771
951880428     2.178845
951880419     2.074932
951880482    10.836942
951880522     1.355534
951880655     6.237453
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

6.984041060757463


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
951879810    4.167651
951880180    5.031256
951880237    7.981599
951880300    4.777099
951880379    3.284454
951880367    3.939439
951880428    1.957011
951880419    1.932125
951880482    3.980739
951880522    0.906494
951880655    2.852386
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

3.7100230286391818


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])